## 无监督学习 (Unsupervised Learning)

相关算法：
1. K-means聚类
2. PCA主成分分析

作业类型：
1. 代码补全
2. 简答题

数据集：
Fashion-MNIST

### Fashion-MNIST数据集(Fashion-MNIST Dataset)

来源：[A MNIST-like fashion product database](https://github.com/zalandoresearch/fashion-mnist)

Fashion-MNIST数据集由[Zalando](https://jobs.zalando.com/tech/)提出，与经典的[MNIST](http://yann.lecun.com/exdb/mnist/)数据集格式完全相同：包含10个种类，总计60000张训练图片和10000张测试图片，每张图片为28\*28尺寸的灰度图片。

我们借助tensorflow的接口读取数据集文件，并实现DataLoader类用于整理数据。

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import random
import numpy as np

class DataLoader:
    def __init__(self, folder):
        self.dataset = input_data.read_data_sets("../input/" + folder)
        self.train_dataset = {}
        self.test_dataset = {}
        for images, labels, dataset in [(self.dataset.train.images, self.dataset.train.labels, self.train_dataset), 
                                        (self.dataset.validation.images, self.dataset.validation.labels, self.train_dataset), 
                                        (self.dataset.test.images, self.dataset.test.labels, self.test_dataset)]:
            assert images.shape[0] == labels.shape[0]
            for i in range(images.shape[0]):
                if not labels[i] in dataset:
                    dataset[labels[i]] = []
                dataset[labels[i]].append(images[i])
        
    def load(self, label_list, train_n=-1, test_n=-1, shuffle=True):
        train_list = []
        test_list = []
        new_label = 0
        for label in label_list:
            for image in self.train_dataset[label]:
                train_list.append((image, new_label))
            for image in self.test_dataset[label]:
                test_list.append((image, new_label))
            new_label += 1
        if shuffle:
            random.shuffle(train_list)
            random.shuffle(test_list)
        if train_n > 0:
            train_list = train_list[: train_n]
        if test_n > 0:
            test_list = train_list[: test_n]
        
        return np.array([pair[0] for pair in train_list]), np.array([pair[1] for pair in train_list]), \
            np.array([pair[0] for pair in test_list]), np.array([pair[1] for pair in test_list]),
                

实例化DataLoader并读取数据集文件。

In [2]:
dataLoader = DataLoader("Fashion_MNIST8667")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../input/Fashion_MNIST8667/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../input/Fashion_MNIST8667/train-labels-idx1-ubyte.gz
Extracting ../input/Fashion_MNIST8667/t10k-images-idx3-ubyte.gz
Extracting ../input/Fashion_MNIST8667/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


读取所有数据，检查数据格式，所有图片均已向量化处理。

In [3]:
X_train, y_train, X_test, y_test = dataLoader.load([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


预览每种类别的部分图片。

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

classes = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Boot']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].reshape((28,28)),cmap=plt.cm.gray)
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

<Figure size 432x288 with 70 Axes>

我们选取其中的5个类别的图片进行K-means聚类学习。

In [5]:
selected_classes = [0, 1, 2, 3, 4]
n = len(selected_classes)

X_train, y_train, X_test, y_test = dataLoader.load(selected_classes)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(30000, 784) (30000,)
(5000, 784) (5000,)


### K-means聚类算法(k-means Clustering)

实现最基本的K-means聚类算法。

算法流程：
1. 提供所需簇(cluster)的数量k。
2. 随机选取k个实例作为种子节点，即作为每个簇的质心(centroid)。
3. 迭代以下步骤：
	* 将每实例分配给最近质心相关联的簇。
	*	重新估计每个簇的质心。
4. 当聚类收敛时停止，或者在经过固定次数的迭代之后。

**TODO：你需要补全K_Means类中fit函数的代码实现**

代码解释：
1. K_Means类中n_clusters变量为算法流程步骤1中的k，centroids为算法流程步骤2中的质心。
2. fit函数参数列表中的max_iter为算法流程步骤4中的最大迭代次数，epsilon为收敛的阈值。

要求：
1. 实现算法流程中的所有步骤，包括质心的随机选取，迭代的收敛控制。
2. 对fit函数的返回值没有特别要求，只需要将质心迭代结果存于centroids中，用于predict函数调用。
3. 对质心的距离函数没有特别要求，可以尝试各种距离函数。

In [6]:
class K_Means:
    def __init__(self, n_clusters=5):
        self.n_clusters = n_clusters
        self.centroids = None
        
    def fit(self, X, max_iter=300, epsilon=0.01):
        n = X.shape[0]
        self.centroids = X[random.sample(range(n), self.n_clusters)]
        old_pred = np.array([-1] * n)

        for t in range(max_iter):
            pred = self.predict(X)
            if np.array_equal(pred, old_pred):
                break
            old_pred = pred
            self.centroids = np.array([np.mean(X[np.where(pred == cluster)[0]], axis=0)
                                       for cluster in range(self.n_clusters)])

    def predict(self, X):
        return np.array([np.argmin(np.diag(np.dot(self.centroids - X[i], (self.centroids - X[i]).T)))
                        for i in range(X.shape[0])])


在测试数据集上进行测试，并输出K-means聚类算法聚类分布。

In [7]:
k_means = K_Means(n_clusters=n)
k_means.fit(X_train, max_iter=300, epsilon=0.001)
y_predicted = k_means.predict(X_test)
result = np.zeros((n, n), dtype=int)
for i in range(X_test.shape[0]):
    result[y_test[i]][y_predicted[i]] += 1
print(result)
result = result * 1.0
for i in range(n):
    result[i] /= np.sum(result[i])
print(result)

[[  7  33 105 305 550]
 [838  14  80  66   2]
 [  3 623  16 346  12]
 [287  20 448 235  10]
 [  8 685 156 151   0]]
[[0.007 0.033 0.105 0.305 0.55 ]
 [0.838 0.014 0.08  0.066 0.002]
 [0.003 0.623 0.016 0.346 0.012]
 [0.287 0.02  0.448 0.235 0.01 ]
 [0.008 0.685 0.156 0.151 0.   ]]


通过可视化直观表现聚类的分布情况。

In [8]:
plt.imshow(result)
plt.colorbar()
plt.show()

<Figure size 432x288 with 2 Axes>

由于K-means聚类算法属于无监督学习算法，我们无法知晓每个质心和真实分类的对应关系，没有直观的正确率概念。这里我们假设最终每个质心分别与真实分类之间是一一对应关系，并通过枚举其对应关系得到一个自定义的正确率。

**最终你实现的K-means聚类算法应该达到50%的正确率。**

最终的学习结果可能存在一定的波动性。

**问题**：对于如何改进K-means聚类算法，例如迭代速度、稳定性、避免局部最优等方面，你有什么想法？
**答案**：在初始的 centroids 选择阶段选第 k 个 centroid 时，与前 k - 1 个 centroids 的距离都更远的点被选为第 k 个 centroid 的概率会更大，这样可以提高算法的稳定性，避免局部最优。在每次迭代要重新计算每个点属于哪个 cluster 时，可以不用计算该点与所有 centroids 的距离，而是可以只计算其与上一轮迭代中与这个点最近的几个点的距离，这样可以加快迭代速度。

**另外我发现一个问题……`K_Means` 类里的 `fit` 函数的 epsilon 似乎用不到……**

In [9]:
from itertools import permutations as perm

score = 0
for p in list(perm([i for i in range(n)])):
    s = 0
    for k in range(n):
        s += result[k][p[k]]
    score = max(score, s)
print(score / np.sum(result))

0.5734


### PCA主成分分析(Principal Components Analysis)

基于特征值分解协方差矩阵方法实现PCA算法。

算法流程：
1. 确定原矩阵$\mathbf{X}_{n \times m}$以及主成分数量k。
2. 对$\mathbf{X}$的每一维去中心化，即减掉各自维度的平均值。
3. 计算协方差矩阵$\frac{1}{n}\mathbf{X}^T\mathbf{X}$的特征值和特征向量。
4. 选取k个最大的特征值对应的特征向量，组成降维投影矩阵。
5. 对原矩阵进行降维处理并输出，维度为$n \times k$。

**TODO：你需要补全pca函数的代码实现**

代码解释：
1. pca函数参数列表中的X和k为PCA主成分分析算法流程的步骤1中的原矩阵$\mathbf{X}_{n \times m}$以及主成分数量k。


要求：
1. 可以使用numpy库中的特征值和特征向量计算函数，但不允许直接调用sklearn库中的PCA相关函数。
2. 可以不用统一每个维度的方差。

In [10]:
def pca(X, k):
    """
    :param X: shape (m, n) with m number of samples, n number of dimensions
    :param k: number of principal components
    :return:
    """
    m, n = X.shape
    X = X - np.mean(X, axis=0)
    cov = np.matmul(X.T, X) / m
    eig_w, eig_v = np.linalg.eig(cov)
    pc = eig_v[:, np.argsort(eig_w)[::-1][:k]]
    projection = np.matmul(X, pc)
    return projection

对PCA主成分分析算法的结果进行可视化处理。

In [11]:
X_input = np.concatenate([X_test, k_means.centroids])
X_pca = pca(X_input, 2)
color = ['r', 'b', 'g', 'y', 'c']
for i in range(n):
    x_list = []
    y_list = []
    for j in range(X_test.shape[0]):
        if y_predicted[j] == i:
            x_list.append(X_pca[j][0])
            y_list.append(X_pca[j][1])
    plt.plot(x_list, y_list, '.', color=color[i])
    plt.plot([X_pca[- n + i][0]], [X_pca[- n + i][1]], 's', color='k')
plt.show()

<Figure size 432x288 with 1 Axes>

与sklearn库中的PCA实现对比。

**最终你实现的PCA主成分分析算法的结果应该与sklearn库的实现相似。**

可能会存在旋转、缩放、镜像等差异，但拓扑关系应该保持一致。

**问题**：对于PCA主成分分析算法进行数据处理的优缺点，你有什么想法？
**答案**：
优点：能够对数据进行降维并实现可视化，能够去除“非主成分”的噪声，算法中没有参数，没有随机性
缺点：主成分相比起原来的维度，缺乏了解释性

In [12]:
from sklearn.decomposition import PCA

X_input = np.concatenate([X_test, k_means.centroids])
X_pca = PCA(2).fit_transform(X_input)
color = ['r', 'b', 'g', 'y', 'c']
for i in range(n):
    x_list = []
    y_list = []
    for j in range(X_test.shape[0]):
        if y_predicted[j] == i:
            x_list.append(X_pca[j][0])
            y_list.append(X_pca[j][1])
    plt.plot(x_list, y_list, '.', color=color[i])
    plt.plot([X_pca[- n + i][0]], [X_pca[- n + i][1]], 's', color='k')
plt.show()

<Figure size 432x288 with 1 Axes>